# COGS 108 - Data Checkpoint

# Names

- Adory Vo
- Andrew Park
- Royce Labuguen
- Schuyler Voss

<a id='research_question'></a>
# Research Question

Do weather conditions have a notable effect on the performance of Major League Baseball players? How are stats such as hits, runs, home runs, pitch velocity, and errors affected when players are put in weather conditions that are more extreme than they are used to?

# Dataset(s)

---
- Dataset Name: 2016 MLB Season
- Link to the dataset: https://www.kaggle.com/datasets/cyaris/2016-mlb-season
- Number of observations: 26

Basic summary data from every game in the 2016 MLB season.
Includes climate, teams, stadium, etc.

---
- Dataset Name: Sportradar Baseball Dataset
- Link to the dataset: https://www.kaggle.com/datasets/sportradar/baseball
- Number of observations: 145

Detailed play-by-play data for every 2016 MLB game. Tracks every pitch, steal, and lineup event for each at-bat.

# Setup

In [ ]:
# Import datasets from Shared Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Function to retrieve file path from Google Drive
def drive_path(filename):
  return f'/content/drive/Shareddrives/COGS 108 - Group 031 Project/{filename}'

Mounted at /content/drive


In [ ]:
# Import essential libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data Cleaning

## 2016 MLB Season

We chose data from 2016 MLB season games to track weather conditions for. Additionally, we are specifically examining the Rays and White Sox, so we want to filter out the weather conditions from their games.

In [83]:
season = pd.read_csv(drive_path('2016-mlb-season.csv'))

In [ ]:
print(season.shape)
season.head()

(2463, 26)


,Unnamed: 0,attendance,away_team,away_team_errors,away_team_hits,away_team_runs,date,field_type,game_type,home_team,...,temperature,wind_speed,wind_direction,sky,total_runs,game_hours_dec,season,home_team_win,home_team_loss,home_team_outcome
0,0,40030.0,New York Mets,1,7,3,2016-04-03,on grass,Night Game,Kansas City Royals,...,74.0,14.0,from Right to Left,Sunny,7,3.216667,regular season,1,0,Win
1,1,21621.0,Philadelphia Phillies,0,5,2,2016-04-06,on grass,Night Game,Cincinnati Reds,...,55.0,24.0,from Right to Left,Overcast,5,2.383333,regular season,1,0,Win
2,2,12622.0,Minnesota Twins,0,5,2,2016-04-06,on grass,Night Game,Baltimore Orioles,...,48.0,7.0,out to Leftfield,Unknown,6,3.183333,regular season,1,0,Win
3,3,18531.0,Washington Nationals,0,8,3,2016-04-06,on grass,Night Game,Atlanta Braves,...,65.0,10.0,from Right to Left,Cloudy,4,2.883333,regular season,0,1,Loss
4,4,18572.0,Colorado Rockies,1,8,4,2016-04-06,on grass,Day Game,Arizona Diamondbacks,...,77.0,0.0,in unknown direction,In Dome,7,2.650000,regular season,0,1,Loss


First, let's look at the columns and decide which ones we want.

In [ ]:
season.columns

Index(['Unnamed: 0', 'attendance', 'away_team', 'away_team_errors',
       'away_team_hits', 'away_team_runs', 'date', 'field_type', 'game_type',
       'home_team', 'home_team_errors', 'home_team_hits', 'home_team_runs',
       'start_time', 'venue', 'day_of_week', 'temperature', 'wind_speed',
       'wind_direction', 'sky', 'total_runs', 'game_hours_dec', 'season',
       'home_team_win', 'home_team_loss', 'home_team_outcome'],
      dtype='object')

### Desired Columns Per Game
For each baseball game, there are a subset of columns that are the most helpful for our analysis as well as some other columns that do not provide any useful information. Of course, we would like to know the home and away teams playing, as well as each of these teams' amount of hits, errors, total runs during the game, as well as which team ended up winning. Also, since our research question revolves around finding the relationship between weather conditions and the overall performance of baseball teams, we include the temperature during the game, the overall weather description (sunny, overcast, rain, etc.), and the overall wind speed/direction throughout the game. 

So, let's filter out these columns from our dataset.

In [68]:
def filter_cols(df):
  """Filters columns from the dataset and returns a new dataframe."""
  GAME_COLS = ['date', 'sky', 'temperature', 'wind_speed', 'wind_direction']
  HOME_COLS = ['home_team', 'home_team_hits', 'home_team_errors', 'home_team_runs', 'home_team_win']
  AWAY_COLS = ['away_team', 'away_team_hits', 'away_team_errors', 'away_team_runs']
  
  DESIRED_COLS = GAME_COLS + HOME_COLS + AWAY_COLS

  new_df = df[DESIRED_COLS]

  return new_df


season = filter_cols(season)

Next, let's examine the values of these columns to make sure they are usable.  

In [ ]:
season['away_team_hits'].unique()   # returns ints
season['home_team_win'].unique()    # returns 0 for a loss and 1 for a win
season['sky'].unique()              # returns strings: "Sunny, Overcast, Unknown, Cloudy, In Dome, Drizzle, Rain, Night"
season['temperature'].unique()      # returns floats
season['wind_speed'].unique()       # returns floats
season['wind_direction'].unique()   # returns strings describing direction relative to the field

array([' from Right to Left', ' out to Leftfield',
       ' in unknown direction', ' from Left to Right',
       ' out to Rightfield', ' in from Leftfield', ' in from Rightfield',
       ' out to Centerfield', ' in from Centerfield'], dtype=object)

### Making sure each row in our dataframe contains useful information
Looking at each unique value that could be stored in each of our columns, most of the columns contain either strings (team that is playing) or ints/floats for weather and baseball statistics. However, the 'sky' column contains a value 'Unknown'. We believe that rows where the 'sky' column contains 'Unknown' do not provide us with any useful information, since we are describing the overall relationship between weather and performance. Thus, we'll remove rows which contain this specific column's value.

Additionally, we can rename the wind direction values to be simpler - "from Right to Left" -> "RtoL" for example.

In [69]:
def clean_values(df):
  """Cleans values from the dataset and returns a new dataframe."""
  # Removing rows with sky == 'Unknown'
  new_df = df[df['sky'] != 'Unknown'].copy()

  # Function to rename a wind direction value
  def rename(value):
    """Renames a wind direction value."""
    # Trim whitespace
    value = value.strip()

    # Rename terms
    value = value.replace('from', '')
    value = value.replace('out', '')
    value = value.replace('field', '')
    value = value.replace('Left', 'L')
    value = value.replace('Center', 'C')
    value = value.replace('Right', 'R')
    value = value.replace('in', 'from')
    value = value.replace('from unknown direction', 'NA')

    # Remove spaces
    value = value.replace(' ', '')

    return value
  
  # Renaming all wind direction values
  new_df['wind_direction'] = new_df['wind_direction'].map(rename)

  return new_df


season = clean_values(season)
print(season['sky'].unique())
print(season['wind_direction'].unique())

['Sunny' 'Overcast' 'Cloudy' 'In Dome' 'Drizzle' 'Rain' 'Night']
['RtoL' 'NA' 'toL' 'toR' 'LtoR' 'fromL' 'fromR' 'toC' 'fromC']


Finally, we can use the `away_team` and `home_team` columns to find our desired games!

In [45]:
TEAM_NAMES = ['Tampa Bay Rays', 'Chicago White Sox']
home_games = season[season['home_team'].isin(TEAM_NAMES)]
away_games = season[season['away_team'].isin(TEAM_NAMES)]

team_games = pd.concat([home_games, away_games])
team_games = team_games.reset_index()
team_games.head()

,index,date,sky,temperature,wind_speed,wind_direction,home_team,home_team_hits,home_team_errors,home_team_runs,home_team_win,away_team,away_team_hits,away_team_errors,away_team_runs
0,6,2016-04-05,In Dome,72.0,0.0,NA,Tampa Bay Rays,7,1,3,1,Toronto Blue Jays,9,0,2
1,36,2016-04-17,In Dome,72.0,0.0,NA,Tampa Bay Rays,8,0,3,1,Chicago White Sox,5,1,2
2,41,2016-04-25,In Dome,72.0,0.0,NA,Tampa Bay Rays,5,0,2,1,Baltimore Orioles,5,0,0
3,52,2016-05-01,In Dome,72.0,0.0,NA,Tampa Bay Rays,4,1,1,0,Toronto Blue Jays,5,0,5
4,75,2016-05-07,Sunny,60.0,16.0,LtoR,Chicago White Sox,13,0,7,1,Minnesota Twins,4,0,2


## Rays & White Sox Games

To approach our research question, we chose to examine the Rays from Tampa Bay (Florida) and the White Sox from Chicago (Illinois). The White Sox should be more used to cold and inclement weather, so they are like a control group while the Rays will be a variable group that we expect to perform differently under harsh conditions.

To start, we downloaded play-by-play data from Rays games and White Sox games as CSV files using this code in a Kaggle notebook.
```py
# Retrieve dataset from Kaggle database
import bq_helper
baseball = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="baseball")

# Select and download Rays games
rays_games_query = "SELECT * FROM `bigquery-public-data.baseball.games_wide` WHERE awayTeamName='Rays' or homeTeamName='Rays'"
rays_games = baseball.query_to_pandas_safe(rays_games_query, max_gb_scanned=2)
rays_games.to_csv('rays-games.csv', index=False)

# Select and download White Sox games
white_sox_games_query = "SELECT * FROM `bigquery-public-data.baseball.games_wide` WHERE awayTeamName='White Sox' or homeTeamName='White Sox'"
white_sox_games = baseball.query_to_pandas_safe(white_sox_games_query, max_gb_scanned=2)
white_sox_games.to_csv('white-sox-games.csv', index=False)

```

These files were then saved in a shared Google Drive to be easily imported from in Google Colab.

In [94]:
GAMES_FILE_NAMES = ['2016-rays-games-2022-10-23.csv', '2016-white-sox-games-2022-10-23.csv']

Now, we may examine the raw data from these datasets.

In [95]:
def raw_games():
  """Returns dataframes from the datasets' raw data."""
  final_dfs = []
  for filename in GAMES_FILE_NAMES:
    df = pd.read_csv(drive_path(filename), dtype={'awayFielder12': str, 'homeFielder12': str})

    final_dfs.append(df)
  
  return final_dfs

In [96]:
# Get raw data
rays, white_sox = raw_games()

In [73]:
print([rays.shape, white_sox.shape])
rays.head()

[(50240, 145), (50260, 145)]


,gameId,seasonId,seasonType,year,startTime,gameStatus,attendance,dayNight,duration,durationMinutes,...,homeBatter4,homeBatter5,homeBatter6,homeBatter7,homeBatter8,homeBatter9,lineupTeamId,lineupPlayerId,lineupPosition,lineupOrder
0,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,6ef878e2-1f2d-4855-a250-2ec7c97f961c,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0
1,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,6ef878e2-1f2d-4855-a250-2ec7c97f961c,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0
2,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,6ef878e2-1f2d-4855-a250-2ec7c97f961c,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0
3,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,6ef878e2-1f2d-4855-a250-2ec7c97f961c,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0
4,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,6ef878e2-1f2d-4855-a250-2ec7c97f961c,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0


There are around 50,000 rows of play-by-play data for each team's games - each row represents a play in a game.  
Also, we have 145 columns - we definitely want to filter out the ones we want now.

In [ ]:
rays.columns.values.tolist()

['gameId',
 'seasonId',
 'seasonType',
 'year',
 'startTime',
 'gameStatus',
 'attendance',
 'dayNight',
 'duration',
 'durationMinutes',
 'awayTeamId',
 'awayTeamName',
 'homeTeamId',
 'homeTeamName',
 'venueId',
 'venueName',
 'venueSurface',
 'venueCapacity',
 'venueCity',
 'venueState',
 'venueZip',
 'venueMarket',
 'venueOutfieldDistances',
 'homeFinalRuns',
 'homeFinalHits',
 'homeFinalErrors',
 'awayFinalRuns',
 'awayFinalHits',
 'awayFinalErrors',
 'homeFinalRunsForInning',
 'awayFinalRunsForInning',
 'inningNumber',
 'inningHalf',
 'inningEventType',
 'inningHalfEventSequenceNumber',
 'description',
 'atBatEventType',
 'atBatEventSequenceNumber',
 'createdAt',
 'updatedAt',
 'status',
 'outcomeId',
 'outcomeDescription',
 'hitterId',
 'hitterLastName',
 'hitterFirstName',
 'hitterWeight',
 'hitterHeight',
 'hitterBatHand',
 'pitcherId',
 'pitcherFirstName',
 'pitcherLastName',
 'pitcherThrowHand',
 'pitchType',
 'pitchTypeDescription',
 'pitchSpeed',
 'pitchZone',
 'pitcherPit

We want specific columns about the game, play, venue, and some other properties. We don't need things like venueZip or all of the extraneous fielders and batters.  

Furthermore, we want our data to include information that we can use to match play data to the corresponding games from our 2016 MLB Season dataset. We can use the date in the `startTime` column and the away/home teams to do this. However, we do not want to perform this complex matching condition every time we want to compare the two sets, so let us create a new column that references the index of the corresponding game in the other dataset.

In [118]:
from datetime import datetime, timedelta

def get_index(row):
  ht = row['homeTeamName'][0]
  at = row['awayTeamName'][0]
  st = row['startTime'][0]
  date = st[:10]

  home_check = team_games[team_games['home_team'].str.contains(ht)]
  
  away_check = home_check[home_check['away_team'].str.contains(at)]

  # Date arithmetic to fix off by one erorr
  date_format = '%Y-%m-%d'
  date_obj = datetime.strptime(date, date_format)
  new_date = date_obj - timedelta(days=1)
  new_date_str = new_date.strftime(date_format)

  date_check = away_check[away_check['date'].str.contains(new_date_str)]

  index = date_check.iloc[0]['index']

  return index

rays.assign(index=get_index).head()

,gameId,seasonId,seasonType,year,startTime,gameStatus,attendance,dayNight,duration,durationMinutes,...,homeBatter5,homeBatter6,homeBatter7,homeBatter8,homeBatter9,lineupTeamId,lineupPlayerId,lineupPosition,lineupOrder,index
0,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0,1794
1,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0,1794
2,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0,1794
3,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0,1794
4,0de78e2b-70a7-4f86-b736-fb0daa18b5a5,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-02 00:15:00+00:00,closed,30554,N,2:45,165,...,d4afcfdc-b875-4163-ad7f-eb26b7228f80,03fe9527-5071-4ae1-b521-0a222e41e581,fc1463b4-7507-4d8b-87c4-0df02b99675e,d36e1643-ef4d-48d6-b898-d8654bc7f9e1,f7deada3-d11e-490e-8658-ae83b7612014,NaN,NaN,0,0,1794


Finally, let's pull the code we wrote to perform all of this cleaning & filter out the columns that we want from both of these datasets.

In [116]:
def get_index(row):
  ht = row['homeTeamName'][0]
  at = row['awayTeamName'][0]
  st = row['startTime'][0]
  date = st[:10]

  home_check = team_games[team_games['home_team'].str.contains(ht)]
  
  away_check = home_check[home_check['away_team'].str.contains(at)]

  # Date arithmetic to fix off by one erorr
  date_format = '%Y-%m-%d'
  date_obj = datetime.strptime(date, date_format)
  new_date = date_obj - timedelta(days=1)
  new_date_str = new_date.strftime(date_format)

  date_check = away_check[away_check['date'].str.contains(new_date_str)]

  index = date_check.iloc[0]['index']

  return index

def clean_games():
  """Cleans datasets and returns dataframes."""
  final_dfs = []
  for filename in GAMES_FILE_NAMES:
    df = pd.read_csv(drive_path(filename), dtype={'awayFielder12': str, 'homeFielder12': str})

    # Clean df
    df = df.head()

    # Add index column
    df = df.assign(index=get_index)
    
    # Filter column names
    GAME_COLS = ['index', 'awayTeamName', 'homeTeamName']
    PLAY_COLS = ['hitterFirstName', 'hitterLastName', 'hitLocation', 'hitType', \
                 'pitcherFirstName', 'pitcherLastName', 'pitchType', 'pitchTypeDescription', 'pitchZone', \
                 'pitchSpeed', 'outcomeDescription']
    VENUE_COLS = ['venueName']
    PROP_COLS = ['is_ab', 'is_ab_over', 'is_hit', 'is_on_base', 'is_bunt', 'is_bunt_shown', 'is_double_play', 'is_triple_play', 'is_wild_pitch', 'is_passed_ball']
    DESIRED_COLS = GAME_COLS + PLAY_COLS + VENUE_COLS + PROP_COLS

    df = df[DESIRED_COLS]

    # Store cleaned df
    final_dfs.append(df)

  return final_dfs

In [117]:
# Get cleaned data
clean_rays, clean_white_sox = clean_games()
clean_rays.head()

,index,awayTeamName,homeTeamName,hitterFirstName,hitterLastName,hitLocation,hitType,pitcherFirstName,pitcherLastName,pitchType,...,is_ab,is_ab_over,is_hit,is_on_base,is_bunt,is_bunt_shown,is_double_play,is_triple_play,is_wild_pitch,is_passed_ball
0,1794,Rays,Royals,Taylor,Motter,0,NaN,Daniel,Duffy,CH,...,0,0,0,0,0,0,0,0,0,0
1,1794,Rays,Royals,Omar,Infante,22,LD,Christopher,Archer,SL,...,1,1,0,0,0,0,0,0,0,0
2,1794,Rays,Royals,Taylor,Motter,0,NaN,Wade,Davis,FA,...,0,0,0,0,0,0,0,0,0,0
3,1794,Rays,Royals,Logan,Morrison,15,NaN,Daniel,Duffy,FA,...,1,1,1,1,0,0,0,0,0,0
4,1794,Rays,Royals,Taylor,Motter,0,NaN,Daniel,Duffy,FA,...,0,0,0,0,0,0,0,0,0,0
